<a href="https://colab.research.google.com/github/Dhaneshkp/DesktopAssistant/blob/main/DATA_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

Everst_data=pd.read_csv('Everest_data.csv')
data=Everst_data
data['Attained_age']=data['Age']+data['Duration']
data['Premium_rate']=data['BasePremiumAmount']/data['BaseNARAmount']*10000
#exclude_cols = ['Gender','RiskClass']
Coulmns_to_keep=['Age','Premium_rate','Gender','RiskClass','Duration']
data=data[data['SingleOrJointType']=="Single"]
#data=data[data['Gender']=="Male"]
#data=data[data['RiskClass']=="NS1"]
data=data[Coulmns_to_keep]
data['index']=data.index
numerical_columns = data.select_dtypes(include=['number']).columns
categorical_columns = data.select_dtypes(include=['object']).columns
data_copy=data.copy()
data

<ipython-input-3-9b3f2a0dd592>:6: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  Everst_data=pd.read_csv('Everest_data.csv')


,Age,Premium_rate,Gender,RiskClass,Duration,index
0,38,3.795200,Male,NS1,3,0
1,31,1.592963,Female,NS1,4,1
2,50,9.488000,Male,NS1,3,2
3,49,15.824000,Female,NS1,8,3
4,53,22.345600,Male,NS1,6,4
...,...,...,...,...,...,...
98914,39,4.321600,Male,NS1,1,98914
98915,26,3.160855,Male,NS1,1,98915
98916,38,2.740398,Male,NS1,1,98916
98917,27,1.062667,Female,NS1,1,98917


In [7]:
# Create a multi-index dataframe using Age, Duration, and a combined Gender and RiskClass column
multi_index_df = data.set_index(['Age', 'Duration', data['Gender'] + '_' + data['RiskClass']])

# Convert the multi-index dataframe to a regular dataframe
result_df = multi_index_df.reset_index().drop(['Gender', 'RiskClass'], axis=1)
print(result_df)


       Age  Duration     level_2  Premium_rate  index
0       38         3    Male_NS1      3.795200      0
1       31         4  Female_NS1      1.592963      1
2       50         3    Male_NS1      9.488000      2
3       49         8  Female_NS1     15.824000      3
4       53         6    Male_NS1     22.345600      4
...    ...       ...         ...           ...    ...
98914   39         1    Male_NS1      4.321600  98914
98915   26         1    Male_NS1      3.160855  98915
98916   38         1    Male_NS1      2.740398  98916
98917   27         1  Female_NS1      1.062667  98917
98918   44         1    Male_NS1      4.744000  98918

[98919 rows x 5 columns]


In [8]:
import numpy as np
import pandas as pd

# Before removing NAs
print("Before removing NAs:")
print(result_df.describe(include='all'))
print(result_df.count())

# Drop rows with missing values
result_df_no_na = result_df.dropna()

# After removing NAs
print("\nAfter removing NAs:")
print(result_df_no_na.describe(include='all'))
print(result_df_no_na.count())


Summary before dropping NA:
                 Age      Duration   level_2  Premium_rate         index
count   98919.000000  98919.000000     98919  98919.000000  98919.000000
unique           NaN           NaN         2           NaN           NaN
top              NaN           NaN  Male_NS1           NaN           NaN
freq             NaN           NaN     69688           NaN           NaN
mean       39.164791      4.353279       NaN      9.567856  49459.000000
std        11.308727      2.642747       NaN     15.255843  28555.599976
min         0.000000      0.000000       NaN    -14.188800      0.000000
25%        31.000000      2.000000       NaN      3.061328  24729.500000
50%        37.000000      4.000000       NaN      4.536000  49459.000000
75%        47.000000      6.000000       NaN      9.027429  74188.500000
max        75.000000     10.000000       NaN    443.505675  98918.000000

Summary after dropping NA:
                 Age      Duration   level_2  Premium_rate         i

In [19]:
# prompt: for teh data ccreate the difference of value of premium rate with the avearege in each frame where frameis oof length  age 5 and duration lengthe of 3

import numpy as np
import pandas as pd

# Assuming 'result_df_no_na' is your DataFrame from the preceding code

# Define frame sizes
age_frame_size = 5
duration_frame_size = 3

# Create a new column for the difference from average premium rate
result_df_no_na['Premium_Diff'] = 0

# Iterate over age and duration frames
for age_start in range(result_df_no_na['Age'].min(), result_df_no_na['Age'].max() + 1, age_frame_size):
  for duration_start in range(result_df_no_na['Duration'].min(), result_df_no_na['Duration'].max() + 1, duration_frame_size):
    age_end = age_start + age_frame_size
    duration_end = duration_start + duration_frame_size

    # Filter data for the current frame
    frame_data = result_df_no_na[
        (result_df_no_na['Age'] >= age_start) & (result_df_no_na['Age'] < age_end) &
        (result_df_no_na['Duration'] >= duration_start) & (result_df_no_na['Duration'] < duration_end)
    ]

    # Calculate the average premium rate for the frame
    avg_premium = frame_data['Premium_rate'].mean()

    # Calculate the difference from average for each row in the frame
    result_df_no_na.loc[frame_data.index, 'Premium_Diff'] = frame_data['Premium_rate'] - avg_premium

print(result_df_no_na.sort_values(by =['level_2','Age','Duration']))


       Age  Duration     level_2  Premium_rate  index  Premium_Diff
7143     0         1  Female_NS1      4.778883   7143      2.148941
18060    0         1  Female_NS1      4.776000  18060      2.146057
67424    0         1  Female_NS1      4.776000  67424      2.146057
68022    0         1  Female_NS1      4.776000  68022      2.146057
68058    0         1  Female_NS1      4.779761  68058      2.149818
...    ...       ...         ...           ...    ...           ...
81298   75         7    Male_NS1    276.891926  81298     61.766448
2337    75         8    Male_NS1    307.348166   2337     92.222688
17769   75         8    Male_NS1    307.345774  17769     92.220296
4167    75         9    Male_NS1    363.103161   4167     80.502351
68227   75         9    Male_NS1    363.097856  68227     80.497046

[98919 rows x 6 columns]


In [23]:
# Calculate average Premium_Diff for duplicates
average_premium_diff = result_df_no_na.groupby(['Age', 'Duration', 'level_2'])['Premium_Diff'].mean().reset_index()

average_premium_diff.pivot(index=['level_2','Age'], columns='Duration', values='Premium_Diff')

      Age  Duration     level_2  Premium_Diff
0       0         1  Female_NS1      2.147578
2       0         2  Female_NS1     -0.079859
4       0         3  Female_NS1      0.281833
6       0         4  Female_NS1     -0.462017
8       0         5  Female_NS1     -0.460850
...   ...       ...         ...           ...
1460   75         5    Male_NS1     56.869759
1462   75         6    Male_NS1      1.780098
1464   75         7    Male_NS1     61.766448
1466   75         8    Male_NS1     92.221492
1468   75         9    Male_NS1     80.499698

[1470 rows x 4 columns]


Duration        0          1          2          3          4          5   \
level_2    Age                                                              
Female_NS1 0   NaN   2.147578  -0.079859   0.281833  -0.462017  -0.460850   
           1   NaN  -0.078743  -0.930564  -0.458333  -0.461400  -0.670564   
           2   NaN  -1.143710  -1.461953  -0.435340  -0.679690        NaN   
           3   NaN  -1.461879  -1.461943  -0.673636  -0.673121  -0.671816   
           4   NaN  -1.461943        NaN  -0.674581  -0.672700  -0.669653   
...             ..        ...        ...        ...        ...        ...   
Male_NS1   71  NaN   5.943010  19.645922  -2.844868  13.410294  37.651128   
           72  NaN  11.740831  26.284982   6.136396  25.108764  48.823387   
           73  NaN  18.381975        NaN  17.519365  40.179516  65.900677   
           74  NaN        NaN  43.782241  27.848979  53.247941  81.918322   
           75  NaN  14.224003  32.669536  -1.738641  25.562183  56.869759   

Duration               6           7           8           9          10  
level_2    Age                                                            
Female_NS1 0    -0.483057   -0.481267   -0.479675   -0.582600  -0.583934  
           1    -0.483325   -0.482520         NaN   -0.580580        NaN  
           2    -0.483084   -0.479562         NaN         NaN  -0.476213  
           3          NaN   -0.481409   -0.481268   -0.478228  -0.373177  
           4    -0.481765   -0.481159   -0.265801         NaN        NaN  
...                   ...         ...         ...         ...        ...  
Male_NS1   71   15.898747   41.631513   68.402944   26.661429  67.090880  
           72   28.346912         NaN   90.432967   62.455813        NaN  
           73   48.898766   80.838863  120.571259  100.607797        NaN  
           74   67.979375  104.665178  148.293467  140.079428        NaN  
           75    1.780098   61.766448   92.221492   80.499698        NaN  

[152 rows x 11 columns]

In [25]:
# prompt: average_premium_diff  : assume its likea picture with 75*10*2 and use a cnn to find the maximun absolute value in each frame of length 5*3*2  and predict the values for maz aboulte of premium_diff

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense

# Reshape the data to fit the CNN input shape (samples, age_frames, duration_frames, gender_risk_frames, channels)
# Assuming 'average_premium_diff' is a NumPy array of shape (75, 10, 2)
average_premium_diff_reshaped = average_premium_diff.reshape(1, 15, 5, 2, 1)

# Define the CNN model
model = Sequential()
model.add(Conv3D(32, (5, 3, 2), activation='relu', input_shape=(15, 5, 2, 1)))
model.add(MaxPooling3D((1, 1, 1)))  # No pooling to preserve frame-wise maximum
model.add(Flatten())
model.add(Dense(1))  # Output a single value (maximum absolute Premium_Diff)

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Mean squared error for regression

# Train the model (you'll need labeled data for this step)
# For demonstration, let's assume 'y_train' contains the true maximum absolute values
# model.fit(average_premium_diff_reshaped, y_train, epochs=10)

# Predict the maximum absolute Premium_Diff for each frame
predictions = model.predict(average_premium_diff_reshaped)

# Reshape predictions to match the original frame structure
predicted_max_abs_diff = predictions.reshape(15, 5)

print(predicted_max_abs_diff)


AttributeError: 'DataFrame' object has no attribute 'reshape'